In [1]:
import sys, subprocess, os
from pathlib import Path

In [2]:
from dataclasses import dataclass

@dataclass
class WidgetConfig:
    widget_type: str
    metric: str
    fromRow: int # Can be passed directly to QGridLayout::addWidget
    fromCol: int # Can be passed directly to QGridLayout::addWidget
    rowSpan: int # Can be passed directly to QGridLayout::addWidget
    colSpan: int # Can be passed directly to QGridLayout::addWidget
    color_scheme: str
    is_separator: bool = False


In [5]:
class LayoutParser:
    def __init__(self, filepath: str):
        self.filepath = filepath
        self.widgets = [] # list of WidgetConfig objects
        self.layout_str = ''
        self.theme_str = ''
        self.row_strings = []
        self.n_rows = 0
        self.n_cols = 0
        self.occupancy_matrix = []
        self.parse_file(filepath)
        self.parse_widgets()

    def parse_file(self, filepath: str):
        """Parse a layout file and store the layout information in class attributes."""
        # Read file contents
        with open(filepath) as f:
            self.layout_str = f.read()

        # Split into rows and handle theme
        if 'theme:' in self.layout_str:
            self.theme_str = self.layout_str.split('theme:')[1].strip().split('\n')[0]
            self.row_strings = self.layout_str.split('\n')[1:]
        else:
            self.theme_str = 'light'
            self.row_strings = self.layout_str.split('\n')

        # Calculate grid dimensions
        self.n_rows = len(self.row_strings)
        
        # Find maximum columns by summing widget widths in each row
        self.n_cols = 1
        for row_str in self.row_strings:
            n_cols_in_row = 0
            for substring in row_str.split('x')[1:]:
                # Catch errors where the 'x' is not adjacent to numbers (e.g. for text widgets).
                try:
                    n_cols_in_row += int(substring[0])
                except:
                    continue
            self.n_cols = max(self.n_cols, n_cols_in_row)
        
        # Create occupancy matrix to keep track of widget positions more easily
        self.occupancy_matrix = [[0]*self.n_cols for _ in range(self.n_rows)]

    def parse_widgets(self):
        """Parse widget configurations from the row strings and store them in self.widgets."""
        # Iterate over all rows:
        for row_idx, row_str in enumerate(self.row_strings):
            widget_strings = [s.strip('[') for s in row_str.split(']')[:-1]]

            # Iterate over all widgets in the row while tracking columns:
            current_col = 0
            for widget_str in widget_strings:
                widget_str = widget_str.rstrip() # removes trailing whitespaces

                # If widget string is empty, it indicates occupancy from above. If so, 
                # increament the column tracker until an empty space is found or row ends.
                if widget_str == '':
                    while self.occupancy_matrix[row_idx][current_col]:
                        current_col += 1
                        if current_col == self.n_cols:
                            break
                    continue

                color = 'A' if 'color' not in widget_str else widget_str.split('color')[-1][-1]
                widget = WidgetConfig(
                    widget_type=widget_str.split(' ')[0],
                    metric='separator' if 'separator' in widget_str else widget_str.split(' ')[1],
                    fromRow=row_idx,
                    fromCol=current_col,
                    rowSpan=int(widget_str.split('x')[-2][-1]), # first char before last 'x'
                    colSpan=int(widget_str.split('x')[-1][-0]), # first char after last 'x'
                    color_scheme=color,
                    is_separator=True if 'separator' in widget_str else False
                )
                self.widgets.append(widget)

                # Populate occupancy matrix:
                for r in range(widget.fromRow, widget.fromRow + widget.rowSpan):
                    for c in range(widget.fromCol, widget.fromCol + widget.colSpan):
                        self.occupancy_matrix[r][c] = 1

                # Skip ahead colSpan columns when tracking current column:
                current_col += widget.colSpan


In [7]:
layout = LayoutParser('settings/default_layout.txt')
print(layout.layout_str)

widgets = layout.widgets
widgets

theme: light
[circle memory 2x2 color a][circle memory 2x1 color a][text ping 1x1]
[                         ][                         ][separator 1x1]
[separator 1x3]


[WidgetConfig(widget_type='circle', metric='memory', fromRow=0, fromCol=0, rowSpan=2, colSpan=2, color_scheme='a', is_separator=False),
 WidgetConfig(widget_type='circle', metric='memory', fromRow=0, fromCol=2, rowSpan=2, colSpan=1, color_scheme='a', is_separator=False),
 WidgetConfig(widget_type='text', metric='ping', fromRow=0, fromCol=3, rowSpan=1, colSpan=1, color_scheme='A', is_separator=False),
 WidgetConfig(widget_type='separator', metric='separator', fromRow=1, fromCol=3, rowSpan=1, colSpan=1, color_scheme='A', is_separator=True),
 WidgetConfig(widget_type='separator', metric='separator', fromRow=2, fromCol=0, rowSpan=1, colSpan=3, color_scheme='A', is_separator=True)]

In [8]:
layout.occupancy_matrix

[[1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 0]]

In [9]:
from ping3 import ping


response_time = ping('8.8.8.8', timeout=2)
if response_time is not None:
    # Convert to milliseconds and round to 1 decimal place
    ms = round(response_time * 1000, 1)

ms

16.7

In [1]:
import time
from ping3 import ping

while True:
    response_time = ping('8.8.8.8', timeout=2)
    if response_time is not None:
        # Convert to milliseconds and round to 1 decimal place
        ms = round(response_time * 1000, 1)
        print(f"Ping: {ms} ms")
    else:
        print("Ping request timed out.")
    time.sleep(1)


Ping: 13.3 ms
Ping: 10.5 ms
Ping: 5.5 ms
Ping: 10.9 ms
Ping: 8.0 ms
Ping: 1.4 ms
Ping: 8.6 ms
Ping: 8.4 ms
Ping: 13.3 ms
Ping: 10.1 ms
Ping: 0.0 ms
Ping: 8.7 ms
Ping: 13.9 ms
Ping: 5.4 ms
Ping: 0.9 ms
Ping: 7.9 ms
Ping: 5.9 ms
Ping: 10.4 ms
Ping: 10.4 ms
Ping: 10.9 ms


KeyboardInterrupt: 

In [2]:
import time
from ping3 import ping

recent_pings = []

while True:
    response_time = ping('8.8.8.8', timeout=2)
    if response_time is not None:
        # Convert to milliseconds and round to 1 decimal place
        ms = round(response_time * 1000, 1)
        recent_pings.append(ms)
        
        # Keep only the 4 most recent pings
        if len(recent_pings) > 4:
            recent_pings.pop(0)
        
        # Calculate the average of the 4 most recent pings
        avg_ping = round(sum(recent_pings) / len(recent_pings), 1)
        print(f"Ping: {avg_ping} ms (average of last {len(recent_pings)} pings)")
    else:
        print("Ping request timed out.")
    time.sleep(1)


Ping: 14.9 ms (average of last 1 pings)
Ping: 17.1 ms (average of last 2 pings)
Ping: 13.2 ms (average of last 3 pings)
Ping: 11.3 ms (average of last 4 pings)
Ping: 8.1 ms (average of last 4 pings)
Ping: 7.4 ms (average of last 4 pings)
Ping: 9.2 ms (average of last 4 pings)
Ping: 7.8 ms (average of last 4 pings)
Ping: 7.2 ms (average of last 4 pings)
Ping: 5.0 ms (average of last 4 pings)
Ping: 12.6 ms (average of last 4 pings)
Ping: 17.2 ms (average of last 4 pings)
Ping: 20.0 ms (average of last 4 pings)
Ping: 21.7 ms (average of last 4 pings)
Ping: 15.0 ms (average of last 4 pings)
Ping: 13.2 ms (average of last 4 pings)


KeyboardInterrupt: 